In [1]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, Conv1D, TimeDistributed, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import backend as K
import time
import numpy as np
import gc
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from sklearn.preprocessing import StandardScaler
import cv2

In [ ]:
# Define Loss Function
def l2_loss(y_true, y_pred):
    diff = y_true - y_pred
    loss = tf.reduce_mean(tf.square(diff))
    return loss

In [ ]:
model = load_model(f'saved_model_cnn_stage0/epoch_732.h5', custom_objects={'l2_loss': l2_loss})

In [ ]:
model.summary()

In [ ]:
# Load Dataset
x_train_gesture  = np.load('traindata_bath_concate/data_bath_train.npy')
gesture_labels_train = np.load('traindata_bath_concate/label_bath_train.npy')
x_test_gesture = np.load('traindata_bath_concate/data_bath_test.npy')
gesture_labels_test = np.load('traindata_bath_concate/label_bath_test.npy')
x_val_gesture = np.load('traindata_bath_concate/data_bath_val.npy')
gesture_labels_val = np.load('traindata_bath_concate/label_bath_val.npy')

print("Gesture data train shape:", np.shape(x_train_gesture))
print("Gesture labels train shape:", np.shape(gesture_labels_train))

label_encoder = LabelEncoder()
gesture_labels_train_encoded = label_encoder.fit_transform(gesture_labels_train)
gesture_labels_test_encoded = label_encoder.transform(gesture_labels_test)
gesture_labels_val_encoded = label_encoder.transform(gesture_labels_val)

gesture_labels_train_one_hot = to_categorical(gesture_labels_train_encoded)
gesture_labels_test_one_hot = to_categorical(gesture_labels_test_encoded)
gesture_labels_val_one_hot = to_categorical(gesture_labels_val_encoded)

gesture_labels_train_one_hot = np.reshape(gesture_labels_train_one_hot, (-1, 1, 8))
gesture_labels_test_one_hot = np.reshape(gesture_labels_test_one_hot, (-1, 1, 8))
gesture_labels_val_one_hot = np.reshape(gesture_labels_val_one_hot, (-1, 1, 8))

print("Reshaped Gesture data train shape:", x_train_gesture.shape)
print("Reshaped Gesture labels train shape:", gesture_labels_train_one_hot.shape)

In [6]:
def AGIprogressBar(count, total,start):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)
    duration=time.time()-start
    print('\r[%s] %s%s ...%s sec' % (bar, percents, '%', duration),end=' ')

In [7]:
indices = np.random.permutation(len(x_train_gesture))
x_train_gesture = np.array(x_train_gesture)
gesture_labels_train_one_hot = np.array(gesture_labels_train_one_hot)

x_train_gesture = x_train_gesture[indices]
gesture_labels_train_one_hot = gesture_labels_train_one_hot[indices]

In [8]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=optimizer, loss=l2_loss, metrics=['accuracy'])

In [ ]:
Batch = 8960
epochs = 100
segment_count = 5
rec = []
st = time.time()
train_accuracies = []
val_accuracies = []
train_losses = []
val_losses = []

for ep in range(epochs):
    print(f'EP: {ep + 1}')
    segment_size = len(x_train_gesture) // segment_count

    for seg in range(segment_count):
        start_idx = seg * segment_size
        end_idx = start_idx + segment_size

        x_segment = x_train_gesture[start_idx:end_idx]
        y_segment = gesture_labels_train_one_hot[start_idx:end_idx]

        indices = np.arange(len(x_segment))
        np.random.shuffle(indices)

        x_segment = x_segment[indices]
        y_segment = y_segment[indices]

        for i in range(len(x_segment) // Batch):
            AGIprogressBar(i, len(x_segment) // Batch, st)
            x_batch = x_segment[i * Batch:(i + 1) * Batch]
            y_batch = y_segment[i * Batch:(i + 1) * Batch]

            model.train_on_batch(x_batch, y_batch)

    train_result = model.evaluate(x_train_gesture, gesture_labels_train_one_hot, batch_size=2400, verbose=0)
    train_loss = train_result[0]
    train_acc = train_result[1]

    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    print(' ')
    print(f'Epoch {ep + 1} Training Loss = {train_loss}')
    print(f'Epoch {ep + 1} Training ACC = {train_acc}')

    val_result = model.evaluate(x_val_gesture, gesture_labels_val_one_hot, batch_size=2400, verbose=0)
    val_loss = val_result[0]
    val_acc = val_result[1]

    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    print(f'Epoch {ep + 1} Validation Loss = {val_loss}')
    print(f'Epoch {ep + 1} Validation ACC = {val_acc}')

    os.makedirs('saved_model_cnn_stage1', exist_ok=True)
    if (ep + 1) % 1 == 0:
        model.save(f'saved_model_cnn_stage1/epoch_{ep + 1}.h5')
        print(f'Model saved at epoch {ep + 1}')
    tf.keras.backend.clear_session()
    gc.collect()
    model = load_model(f'saved_model_cnn_stage1/epoch_{ep + 1}.h5', custom_objects={'l2_loss': l2_loss})


best_epoch_accuracy = val_accuracies.index(max(val_accuracies))
best_val_accuracy = val_accuracies[best_epoch_accuracy]
print(f"The best epoch based on validation accuracy is: {best_epoch_accuracy + 1}, with accuracy: {best_val_accuracy:.4f}")

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'])

plt.subplot(1, 2, 2)
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'])

plt.show()


In [ ]:
model = load_model('saved_model_cnn_stage1/epoch_100.h5', custom_objects={'l2_loss': l2_loss})

In [6]:
# hook BN Layer
hook=[]
id=[10]
for i in range(len(id)):
  hook.append(model.layers[id[i]].output)
ModelExtract = Model(inputs=model.input, outputs=hook)


In [ ]:
ModelExtract.output

In [ ]:
# Save prediction results
os.mkdir('Train_Dataset_For_LSTM')
predictions = ModelExtract.predict(x_train_gesture)
labels = np.squeeze(gesture_labels_train_one_hot, axis=1)
np.save('Train_Dataset_For_LSTM/predictions_train.npy', predictions)
np.save('Train_Dataset_For_LSTM/predictions_labels_train', labels)

predictions = ModelExtract.predict(x_val_gesture)
labels = np.squeeze(gesture_labels_val_one_hot, axis=1)
np.save('Train_Dataset_For_LSTM/predictions_val.npy', predictions)
np.save('Train_Dataset_For_LSTM/predictions_labels_val', labels)

predictions = ModelExtract.predict(x_test_gesture)
labels = np.squeeze(gesture_labels_test_one_hot, axis=1)
np.save('Train_Dataset_For_LSTM/predictions_test.npy', predictions)
np.save('Train_Dataset_For_LSTM/predictions_labels_test', labels)